In [3]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import requests
import os

In [85]:
driver = webdriver.Safari()
years = range(1991, 2023)

In [86]:
advanced_stats_url = 'https://www.basketball-reference.com/leagues/NBA_{}_advanced.html'

In [87]:
for year in years:
    with open('files/advanced_stats/{}.html'.format(year), 'w+') as file:
        driver.get(advanced_stats_url.format(year))
        driver.execute_script('window.scrollTo(1, 10000)')
        time.sleep(2)
        html = driver.page_source
        file.write(html)

InvalidSessionIdException: Message: 


In [9]:
df = []

for year in years:
    with open('files/advanced_stats/{}.html'.format(year), 'r') as file:
        file = file.read()
        soup = BeautifulSoup(file, 'html.parser')
        soup.find('tr', class_ = 'thead').decompose()
        advanced_stats = soup.find(id = 'advanced_stats')
        advanced_stats = pd.read_html(str(advanced_stats))[0]
        advanced_stats['Year'] = year
    
        df.append(advanced_stats)

In [88]:
data = pd.concat(df)

In [89]:
data.columns

Index(['Rk', 'Player', 'Pos', 'Age', 'Tm', 'G', 'MP', 'PER', 'TS%', '3PAr',
       'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%',
       'Unnamed: 19', 'OWS', 'DWS', 'WS', 'WS/48', 'Unnamed: 24', 'OBPM',
       'DBPM', 'BPM', 'VORP', 'Year'],
      dtype='object')

In [90]:
data = data.drop(['Rk', 'Unnamed: 19', 'Unnamed: 24'], axis = 1)

In [91]:
data

,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,...,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,Year
0,Alaa Abdelnaby,PF,22,POR,43,290,13.1,.499,.000,.379,...,22.1,0.0,0.5,0.5,.079,-3.4,-1.2,-4.6,-0.2,1991
1,Mahmoud Abdul-Rauf,PG,21,DEN,67,1505,12.2,.448,.099,.097,...,27.2,-0.7,-0.3,-1.0,-0.031,-2.0,-3.0,-5.0,-1.1,1991
2,Mark Acres,C,28,ORL,68,1313,9.2,.551,.014,.472,...,9.3,1.4,1.1,2.5,.090,-2.8,-0.2,-3.0,-0.3,1991
3,Michael Adams,PG,28,DEN,66,2346,22.3,.530,.397,.372,...,28.5,5.8,0.4,6.3,.128,6.0,-0.7,5.3,4.3,1991
4,Mark Aguirre,SF,31,DET,78,2006,16.7,.526,.086,.349,...,25.7,2.8,2.7,5.5,.132,1.2,0.2,1.4,1.7,1991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
836,Thaddeus Young,PF,33,TOR,26,475,15.8,.526,.299,.188,...,16.0,0.5,0.8,1.3,.127,-0.2,2.2,2.0,0.5,2022
837,Trae Young,PG,23,ATL,76,2652,25.4,.603,.395,.358,...,34.4,9.0,1.0,10.0,.181,7.1,-2.0,5.2,4.8,2022
838,Omer Yurtseven,C,23,MIA,56,706,17.4,.546,.045,.247,...,19.9,0.8,1.4,2.1,.145,-1.4,0.4,-1.0,0.2,2022
839,Cody Zeller,C,29,POR,27,355,17.2,.627,.044,.544,...,15.9,0.9,0.2,1.1,.143,-1.2,-1.0,-2.1,0.0,2022


In [92]:
def total_stats(df):
    if df.shape[0] == 1:
        return df
    else:
        row = df[df.get('Tm') == 'TOT']
        row['Tm'] = df.iloc[-1,:]['Tm']
        return row
    
advanced_stats = data.groupby(['Player', 'Year']).apply(total_stats)

In [93]:
advanced_stats

Player Pos Age   Tm   G    MP   PER   TS%  \
Player         Year                                                          
A.C. Green     1991 164      A.C. Green  PF  27  LAL  82  2164  13.8  .556   
               1992 174      A.C. Green  PF  28  LAL  82  2902  16.7  .556   
               1993 157      A.C. Green  PF  29  LAL  82  2819  16.3  .603   
               1994 177      A.C. Green  PF  30  PHO  82  2825  17.0  .555   
               1995 165      A.C. Green  SF  31  PHO  82  2687  14.2  .596   
...                                 ...  ..  ..  ...  ..   ...   ...   ...   
Željko Rebrača 2002 394  Željko Rebrača   C  29  DET  74  1179  16.2  .569   
               2003 374  Željko Rebrača   C  30  DET  30   488  14.0  .596   
               2004 452  Željko Rebrača   C  31  ATL  24   273  10.5  .504   
               2005 446  Željko Rebrača   C  32  LAC  58   928  14.0  .625   
               2006 433  Željko Rebrača   C  33  LAC  29   412  10.7  .592   

                         3PAr   FTr  ...  USG%  OWS  DWS   WS WS/48  OBPM  \
Player         Year                  ...                                    
A.C. Green     1991 164  .101  .557  ...  15.6  3.2  2.9  6.1  .135   0.1   
               1992 174  .070  .569  ...  16.8  5.8  2.9  8.8  .145   1.7   
               1993 157  .065  .531  ...  15.0  6.1  2.6  8.6  .147   1.6   
               1994 177  .038  .391  ...  17.6  6.7  2.6  9.3  .157   1.8   
               1995 165  .206  .556  ...  14.2  4.6  2.1  6.7  .120   0.8   
...                       ...   ...  ...   ...  ...  ...  ...   ...   ...   
Željko Rebrača 2002 394  .000  .468  ...  21.4  1.4  1.9  3.3  .134  -2.9   
               2003 374  .000  .331  ...  19.2  0.7  0.7  1.3  .133  -2.8   
               2004 452  .000  .390  ...  18.2  0.1  0.4  0.5  .097  -4.5   
               2005 446  .000  .363  ...  16.0  1.4  0.9  2.4  .122  -1.7   
               2006 433  .000  .427  ...  15.5  0.2  0.5  0.7  .086  -3.7   

                         DBPM   BPM  VORP  Year  
Player         Year                              
A.C. Green     1991 164  -0.2   0.0   1.1  1991  
               1992 174  -1.0   0.7   2.0  1992  
               1993 157  -0.9   0.7   1.9  1993  
               1994 177  -1.2   0.6   1.8  1994  
               1995 165  -0.9  -0.1   1.3  1995  
...                       ...   ...   ...   ...  
Željko Rebrača 2002 394   0.6  -2.3  -0.1  2002  
               2003 374   0.2  -2.6  -0.1  2003  
               2004 452   0.6  -3.9  -0.1  2004  
               2005 446   0.4  -1.3   0.2  2005  
               2006 433   0.9  -2.8  -0.1  2006  

[14697 rows x 27 columns]

In [94]:
advanced_stats.index = advanced_stats.index.droplevel()
advanced_stats.index = advanced_stats.index.droplevel()

In [95]:
advanced_stats

,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,...,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,Year
164,A.C. Green,PF,27,LAL,82,2164,13.8,.556,.101,.557,...,15.6,3.2,2.9,6.1,.135,0.1,-0.2,0.0,1.1,1991
174,A.C. Green,PF,28,LAL,82,2902,16.7,.556,.070,.569,...,16.8,5.8,2.9,8.8,.145,1.7,-1.0,0.7,2.0,1992
157,A.C. Green,PF,29,LAL,82,2819,16.3,.603,.065,.531,...,15.0,6.1,2.6,8.6,.147,1.6,-0.9,0.7,1.9,1993
177,A.C. Green,PF,30,PHO,82,2825,17.0,.555,.038,.391,...,17.6,6.7,2.6,9.3,.157,1.8,-1.2,0.6,1.8,1994
165,A.C. Green,SF,31,PHO,82,2687,14.2,.596,.206,.556,...,14.2,4.6,2.1,6.7,.120,0.8,-0.9,-0.1,1.3,1995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,Željko Rebrača,C,29,DET,74,1179,16.2,.569,.000,.468,...,21.4,1.4,1.9,3.3,.134,-2.9,0.6,-2.3,-0.1,2002
374,Željko Rebrača,C,30,DET,30,488,14.0,.596,.000,.331,...,19.2,0.7,0.7,1.3,.133,-2.8,0.2,-2.6,-0.1,2003
452,Željko Rebrača,C,31,ATL,24,273,10.5,.504,.000,.390,...,18.2,0.1,0.4,0.5,.097,-4.5,0.6,-3.9,-0.1,2004
446,Željko Rebrača,C,32,LAC,58,928,14.0,.625,.000,.363,...,16.0,1.4,0.9,2.4,.122,-1.7,0.4,-1.3,0.2,2005


In [96]:
player_stats = pd.read_csv('files/player_analysis_files/player_stats.csv')

In [97]:
player_stats.columns

Index(['Unnamed: 0', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG',
       'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT',
       'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
       'PTS', 'Year', 'Pts Won', 'Pts Max', 'Share', 'Team', 'Wins', 'Losses',
       'W/L%', 'Game Score'],
      dtype='object')

In [98]:
player_stats

,Unnamed: 0,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,PTS,Year,Pts Won,Pts Max,Share,Team,Wins,Losses,W/L%,Game Score
0,0,A.C. Green,PF,27,LAL,82,21,26.4,3.1,6.6,...,9.1,1991,0.0,0.0,0.0,Los Angeles Lakers,58,24,0.707,8.79
1,1,Byron Scott,SG,29,LAL,82,82,32.1,6.1,12.8,...,14.5,1991,0.0,0.0,0.0,Los Angeles Lakers,58,24,0.707,10.55
2,2,Elden Campbell,PF,22,LAL,52,0,7.3,1.1,2.4,...,2.8,1991,0.0,0.0,0.0,Los Angeles Lakers,58,24,0.707,2.54
3,3,Irving Thomas,PF,25,LAL,26,0,4.2,0.7,1.9,...,1.8,1991,0.0,0.0,0.0,Los Angeles Lakers,58,24,0.707,1.05
4,4,James Worthy,SF,29,LAL,78,74,38.6,9.2,18.7,...,21.4,1991,0.0,0.0,0.0,Los Angeles Lakers,58,24,0.707,16.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14692,14692,Spencer Hawes,PF,28,MIL,54,1,14.8,2.5,5.1,...,6.2,2017,0.0,0.0,0.0,Milwaukee Bucks,42,40,0.512,5.35
14693,14693,Steve Novak,PF,33,MIL,8,0,2.8,0.3,0.9,...,0.6,2017,0.0,0.0,0.0,Milwaukee Bucks,42,40,0.512,0.17
14694,14694,Terrence Jones,PF,25,MIL,54,12,23.5,4.3,9.1,...,10.8,2017,0.0,0.0,0.0,Milwaukee Bucks,42,40,0.512,9.61
14695,14695,Thon Maker,C,19,MIL,57,34,9.9,1.5,3.2,...,4.0,2017,0.0,0.0,0.0,Milwaukee Bucks,42,40,0.512,3.29


In [99]:
player_stats = player_stats.drop(['Unnamed: 0'], axis = 1)

In [100]:
advanced_stats = advanced_stats.apply(pd.to_numeric, errors = 'ignore')

In [101]:
advanced_stats['Player'] = advanced_stats['Player'].apply(lambda x: x.strip('*'))

In [102]:
player_stats.columns

Index(['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year',
       'Pts Won', 'Pts Max', 'Share', 'Team', 'Wins', 'Losses', 'W/L%',
       'Game Score'],
      dtype='object')

In [103]:
advanced_stats.columns

Index(['Player', 'Pos', 'Age', 'Tm', 'G', 'MP', 'PER', 'TS%', '3PAr', 'FTr',
       'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS',
       'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP', 'Year'],
      dtype='object')

In [104]:
advanced_stats

,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,...,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,Year
164,A.C. Green,PF,27,LAL,82,2164,13.8,0.556,0.101,0.557,...,15.6,3.2,2.9,6.1,0.135,0.1,-0.2,0.0,1.1,1991
174,A.C. Green,PF,28,LAL,82,2902,16.7,0.556,0.070,0.569,...,16.8,5.8,2.9,8.8,0.145,1.7,-1.0,0.7,2.0,1992
157,A.C. Green,PF,29,LAL,82,2819,16.3,0.603,0.065,0.531,...,15.0,6.1,2.6,8.6,0.147,1.6,-0.9,0.7,1.9,1993
177,A.C. Green,PF,30,PHO,82,2825,17.0,0.555,0.038,0.391,...,17.6,6.7,2.6,9.3,0.157,1.8,-1.2,0.6,1.8,1994
165,A.C. Green,SF,31,PHO,82,2687,14.2,0.596,0.206,0.556,...,14.2,4.6,2.1,6.7,0.120,0.8,-0.9,-0.1,1.3,1995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,Željko Rebrača,C,29,DET,74,1179,16.2,0.569,0.000,0.468,...,21.4,1.4,1.9,3.3,0.134,-2.9,0.6,-2.3,-0.1,2002
374,Željko Rebrača,C,30,DET,30,488,14.0,0.596,0.000,0.331,...,19.2,0.7,0.7,1.3,0.133,-2.8,0.2,-2.6,-0.1,2003
452,Željko Rebrača,C,31,ATL,24,273,10.5,0.504,0.000,0.390,...,18.2,0.1,0.4,0.5,0.097,-4.5,0.6,-3.9,-0.1,2004
446,Željko Rebrača,C,32,LAC,58,928,14.0,0.625,0.000,0.363,...,16.0,1.4,0.9,2.4,0.122,-1.7,0.4,-1.3,0.2,2005


Although advanced stats and player stats mostly had different columns, of the ones that were the same, most had the same values. For instance, in different years, A.C. Green played with the Lakers and those are also replicated both the player and advanced stats. However, the advanced_stats also show total minutes played instead of MPG(mintues per game), which means we have to either make it a new column or get rid of it. I'm opting to get rid of it, since we already have a variable that implies how many minutes a player plays throughout a season, those being the 'MP'(MPG) and the percentage of games that a player could've played in their season

In [105]:
advanced_stats = advanced_stats.drop(['MP'], axis = 1)

In [106]:
player_stats

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,PTS,Year,Pts Won,Pts Max,Share,Team,Wins,Losses,W/L%,Game Score
0,A.C. Green,PF,27,LAL,82,21,26.4,3.1,6.6,0.476,...,9.1,1991,0.0,0.0,0.0,Los Angeles Lakers,58,24,0.707,8.79
1,Byron Scott,SG,29,LAL,82,82,32.1,6.1,12.8,0.477,...,14.5,1991,0.0,0.0,0.0,Los Angeles Lakers,58,24,0.707,10.55
2,Elden Campbell,PF,22,LAL,52,0,7.3,1.1,2.4,0.455,...,2.8,1991,0.0,0.0,0.0,Los Angeles Lakers,58,24,0.707,2.54
3,Irving Thomas,PF,25,LAL,26,0,4.2,0.7,1.9,0.340,...,1.8,1991,0.0,0.0,0.0,Los Angeles Lakers,58,24,0.707,1.05
4,James Worthy,SF,29,LAL,78,74,38.6,9.2,18.7,0.492,...,21.4,1991,0.0,0.0,0.0,Los Angeles Lakers,58,24,0.707,16.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14692,Spencer Hawes,PF,28,MIL,54,1,14.8,2.5,5.1,0.484,...,6.2,2017,0.0,0.0,0.0,Milwaukee Bucks,42,40,0.512,5.35
14693,Steve Novak,PF,33,MIL,8,0,2.8,0.3,0.9,0.286,...,0.6,2017,0.0,0.0,0.0,Milwaukee Bucks,42,40,0.512,0.17
14694,Terrence Jones,PF,25,MIL,54,12,23.5,4.3,9.1,0.470,...,10.8,2017,0.0,0.0,0.0,Milwaukee Bucks,42,40,0.512,9.61
14695,Thon Maker,C,19,MIL,57,34,9.9,1.5,3.2,0.459,...,4.0,2017,0.0,0.0,0.0,Milwaukee Bucks,42,40,0.512,3.29


In [113]:
advanced_stats

,Player,Pos,Age,Tm,G,PER,TS%,3PAr,FTr,ORB%,...,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,Year
164,A.C. Green,PF,27,LAL,82,13.8,0.556,0.101,0.557,11.3,...,15.6,3.2,2.9,6.1,0.135,0.1,-0.2,0.0,1.1,1991
174,A.C. Green,PF,28,LAL,82,16.7,0.556,0.070,0.569,11.8,...,16.8,5.8,2.9,8.8,0.145,1.7,-1.0,0.7,2.0,1992
157,A.C. Green,PF,29,LAL,82,16.3,0.603,0.065,0.531,11.5,...,15.0,6.1,2.6,8.6,0.147,1.6,-0.9,0.7,1.9,1993
177,A.C. Green,PF,30,PHO,82,17.0,0.555,0.038,0.391,11.1,...,17.6,6.7,2.6,9.3,0.157,1.8,-1.2,0.6,1.8,1994
165,A.C. Green,SF,31,PHO,82,14.2,0.596,0.206,0.556,8.3,...,14.2,4.6,2.1,6.7,0.120,0.8,-0.9,-0.1,1.3,1995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,Željko Rebrača,C,29,DET,74,16.2,0.569,0.000,0.468,8.4,...,21.4,1.4,1.9,3.3,0.134,-2.9,0.6,-2.3,-0.1,2002
374,Željko Rebrača,C,30,DET,30,14.0,0.596,0.000,0.331,6.5,...,19.2,0.7,0.7,1.3,0.133,-2.8,0.2,-2.6,-0.1,2003
452,Željko Rebrača,C,31,ATL,24,10.5,0.504,0.000,0.390,9.9,...,18.2,0.1,0.4,0.5,0.097,-4.5,0.6,-3.9,-0.1,2004
446,Željko Rebrača,C,32,LAC,58,14.0,0.625,0.000,0.363,6.5,...,16.0,1.4,0.9,2.4,0.122,-1.7,0.4,-1.3,0.2,2005


In [116]:
combined = player_stats.merge(advanced_stats, how = 'outer')

In [117]:
combined[combined['Player'] == 'James Worthy']

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP
4,James Worthy,SF,29,LAL,78,74,38.6,9.2,18.7,0.492,...,7.5,24.6,5.5,3.5,9.0,0.143,2.2,0.3,2.5,3.4
19,James Worthy,SF,30,LAL,54,54,39.0,8.3,18.6,0.447,...,10.4,25.4,1.4,1.6,3.1,0.070,1.5,-0.4,1.1,1.6
36,James Worthy,SF,31,LAL,82,69,28.8,6.2,13.9,0.447,...,10.0,24.9,1.4,1.6,3.0,0.060,0.8,-1.0,-0.2,1.0
8833,James Worthy,SF,32,LAL,80,2,20.0,4.3,10.5,0.406,...,9.8,26.2,-0.5,0.5,0.1,0.002,-0.6,-2.3,-2.9,-0.4


In [120]:
combined.to_csv('files/player_analysis_files/full_player_stats.csv')

In [4]:
combined = pd.read_csv('files/player_analysis_files/full_player_stats.csv')

In [9]:
combined['3P%'] = combined['3P%'].fillna(0)

In [11]:
combined = combined.drop(['Unnamed: 0'], axis = 1)

In [12]:
combined.to_csv('files/player_analysis_files/full_player_stats.csv')

In [16]:
combined = combined.fillna(0)